<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/colab/gensim_lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/usr/local/lib/python3.7/dist-packages/scipy/io/matlab/mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0

11314

In [2]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [3]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

/usr/local/lib/python3.7/dist-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [8]:
from gensim import corpora

dictionary = corpora.Dictionary(tokenized_doc) # (word_id, word_frequency) 의 묶음으로..
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력

/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [20]:
print(dictionary[66])

faith


In [21]:
len(dictionary)

64281

In [9]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.010*"government" + 0.008*"people" + 0.006*"would" + 0.005*"state"')
(1, '0.019*"myers" + 0.010*"candida" + 0.008*"netcom" + 0.006*"offset"')
(2, '0.032*"drive" + 0.019*"scsi" + 0.019*"disk" + 0.015*"system"')
(3, '0.024*"church" + 0.022*"jesus" + 0.014*"christ" + 0.010*"father"')
(4, '0.012*"university" + 0.010*"health" + 0.010*"april" + 0.008*"medical"')
(5, '0.029*"period" + 0.017*"play" + 0.016*"power" + 0.010*"scorer"')
(6, '0.027*"price" + 0.016*"sale" + 0.014*"please" + 0.014*"offer"')
(7, '0.017*"bike" + 0.013*"cars" + 0.013*"engine" + 0.009*"water"')
(8, '0.017*"pain" + 0.012*"cross" + 0.009*"colormap" + 0.009*"mask"')
(9, '0.017*"game" + 0.016*"team" + 0.014*"year" + 0.011*"games"')
(10, '0.011*"windows" + 0.008*"using" + 0.007*"thanks" + 0.007*"window"')
(11, '0.018*"would" + 0.013*"like" + 0.012*"know" + 0.012*"think"')
(12, '0.020*"file" + 0.013*"output" + 0.013*"entry" + 0.009*"chip"')
(13, '0.026*"ground" + 0.023*"wire" + 0.015*"wiring" + 0.015*"neutral"')
(14, '0.

(0, '0.007*"books" + 0.006*"shell" + 0.006*"good" + 0.005*"greek"')
(1, '0.015*"game" + 0.014*"team" + 0.010*"games" + 0.010*"play"')
(2, '0.018*"armenian" + 0.016*"armenians" + 0.013*"jews" + 0.013*"turkish"')
(3, '0.011*"available" + 0.007*"version" + 0.007*"program" + 0.007*"software"')
(4, '0.006*"high" + 0.005*"time" + 0.005*"used" + 0.005*"much"')
(5, '0.010*"nrhj" + 0.007*"wwiz" + 0.006*"bxom" + 0.006*"gizw"')
(6, '0.013*"health" + 0.011*"medical" + 0.008*"food" + 0.007*"pain"')
(7, '0.011*"people" + 0.011*"would" + 0.007*"think" + 0.006*"believe"')
(8, '0.010*"would" + 0.010*"people" + 0.006*"government" + 0.005*"right"')
(9, '0.014*"water" + 0.007*"byte" + 0.006*"bits" + 0.005*"compressed"')
(10, '0.007*"auto" + 0.007*"autos" + 0.006*"mask" + 0.006*"pitcher"')
(11, '0.025*"drive" + 0.015*"scsi" + 0.014*"disk" + 0.011*"hard"')
(12, '0.032*"space" + 0.010*"nasa" + 0.007*"launch" + 0.006*"earth"')
(13, '0.038*"file" + 0.027*"entry" + 0.026*"output" + 0.014*"program"')
(14, '0.010*"information" + 0.008*"encryption" + 0.007*"chip" + 0.007*"system"')
(15, '0.005*"zionist" + 0.004*"portable" + 0.004*"campbell" + 0.004*"seagate"')
(16, '0.016*"like" + 0.016*"would" + 0.013*"know" + 0.011*"good"')
(17, '0.025*"israel" + 0.016*"israeli" + 0.008*"arab" + 0.006*"palestinian"')
(18, '0.019*"windows" + 0.013*"problem" + 0.013*"thanks" + 0.010*"window"')
(19, '0.011*"jesus" + 0.005*"shall" + 0.005*"christ" + 0.005*"spirit"')

In [24]:
print(ldamodel.print_topics())

[(0, '0.007*"books" + 0.006*"shell" + 0.006*"good" + 0.005*"greek" + 0.005*"colormap" + 0.004*"aspi" + 0.004*"cure" + 0.004*"text" + 0.003*"decenso" + 0.003*"english"'), (1, '0.015*"game" + 0.014*"team" + 0.010*"games" + 0.010*"play" + 0.009*"season" + 0.008*"year" + 0.007*"players" + 0.007*"hockey" + 0.007*"first" + 0.007*"league"'), (2, '0.018*"armenian" + 0.016*"armenians" + 0.013*"jews" + 0.013*"turkish" + 0.008*"people" + 0.008*"turkey" + 0.006*"greek" + 0.006*"armenia" + 0.006*"said" + 0.006*"turks"'), (3, '0.011*"available" + 0.007*"version" + 0.007*"program" + 0.007*"software" + 0.007*"also" + 0.007*"file" + 0.007*"image" + 0.006*"data" + 0.006*"files" + 0.005*"graphics"'), (4, '0.006*"high" + 0.005*"time" + 0.005*"used" + 0.005*"much" + 0.005*"power" + 0.005*"also" + 0.005*"like" + 0.005*"work" + 0.004*"would" + 0.004*"good"'), (5, '0.010*"nrhj" + 0.007*"wwiz" + 0.006*"bxom" + 0.006*"gizw" + 0.006*"picture" + 0.005*"sleeve" + 0.005*"tbxn" + 0.005*"bhjn" + 0.005*"bxlt" + 0.004*

[(0, '0.007*"books" + 0.006*"shell" + 0.006*"good" + 0.005*"greek" + 0.005*"colormap" + 0.004*"aspi" + 0.004*"cure" + 0.004*"text" + 0.003*"decenso" + 0.003*"english"'), (1, '0.015*"game" + 0.014*"team" + 0.010*"games" + 0.010*"play" + 0.009*"season" + 0.008*"year" + 0.007*"players" + 0.007*"hockey" + 0.007*"first" + 0.007*"league"'), (2, '0.018*"armenian" + 0.016*"armenians" + 0.013*"jews" + 0.013*"turkish" + 0.008*"people" + 0.008*"turkey" + 0.006*"greek" + 0.006*"armenia" + 0.006*"said" + 0.006*"turks"'), (3, '0.011*"available" + 0.007*"version" + 0.007*"program" + 0.007*"software" + 0.007*"also" + 0.007*"file" + 0.007*"image" + 0.006*"data" + 0.006*"files" + 0.005*"graphics"'), (4, '0.006*"high" + 0.005*"time" + 0.005*"used" + 0.005*"much" + 0.005*"power" + 0.005*"also" + 0.005*"like" + 0.005*"work" + 0.004*"would" + 0.004*"good"'), (5, '0.010*"nrhj" + 0.007*"wwiz" + 0.006*"bxom" + 0.006*"gizw" + 0.006*"picture" + 0.005*"sleeve" + 0.005*"tbxn" + 0.005*"bhjn" + 0.005*"bxlt" + 0.004*"norton"'), (6, '0.013*"health" + 0.011*"medical" + 0.008*"food" + 0.007*"pain" + 0.007*"disease" + 0.006*"patients" + 0.006*"among" + 0.006*"doctor" + 0.004*"university" + 0.004*"years"'), (7, '0.011*"people" + 0.011*"would" + 0.007*"think" + 0.006*"believe" + 0.006*"even" + 0.006*"know" + 0.005*"many" + 0.005*"like" + 0.004*"time" + 0.004*"also"'), (8, '0.010*"would" + 0.010*"people" + 0.006*"government" + 0.005*"right" + 0.005*"think" + 0.005*"state" + 0.004*"president" + 0.004*"well" + 0.004*"going" + 0.004*"make"'), (9, '0.014*"water" + 0.007*"byte" + 0.006*"bits" + 0.005*"compressed" + 0.005*"compression" + 0.004*"compress" + 0.004*"maxbyte" + 0.004*"push" + 0.004*"retail" + 0.003*"solaris"'), (10, '0.007*"auto" + 0.007*"autos" + 0.006*"mask" + 0.006*"pitcher" + 0.006*"section" + 0.006*"cview" + 0.005*"bytes" + 0.004*"dept" + 0.004*"mydisplay" + 0.004*"main"'), (11, '0.025*"drive" + 0.015*"scsi" + 0.014*"disk" + 0.011*"hard" + 0.009*"system" + 0.009*"card" + 0.009*"sale" + 0.008*"drives" + 0.008*"price" + 0.008*"controller"'), (12, '0.032*"space" + 0.010*"nasa" + 0.007*"launch" + 0.006*"earth" + 0.006*"satellite" + 0.005*"shuttle" + 0.005*"orbit" + 0.005*"mission" + 0.004*"lunar" + 0.004*"moon"'), (13, '0.038*"file" + 0.027*"entry" + 0.026*"output" + 0.014*"program" + 0.011*"build" + 0.010*"rules" + 0.009*"section" + 0.009*"oname" + 0.009*"check" + 0.009*"title"'), (14, '0.010*"information" + 0.008*"encryption" + 0.007*"chip" + 0.007*"system" + 0.007*"public" + 0.006*"mail" + 0.006*"keys" + 0.006*"clipper" + 0.006*"security" + 0.005*"number"'), (15, '0.005*"zionist" + 0.004*"portable" + 0.004*"campbell" + 0.004*"seagate" + 0.004*"lopez" + 0.003*"nords" + 0.003*"wales" + 0.003*"packaging" + 0.003*"leagues" + 0.003*"fielding"'), (16, '0.016*"like" + 0.016*"would" + 0.013*"know" + 0.011*"good" + 0.009*"think" + 0.007*"time" + 0.007*"could" + 0.007*"well" + 0.007*"really" + 0.006*"anyone"'), (17, '0.025*"israel" + 0.016*"israeli" + 0.008*"arab" + 0.006*"palestinian" + 0.006*"wiring" + 0.005*"neutral" + 0.005*"south" + 0.005*"navy" + 0.005*"electrical" + 0.005*"arabs"'), (18, '0.019*"windows" + 0.013*"problem" + 0.013*"thanks" + 0.010*"window" + 0.010*"using" + 0.009*"would" + 0.009*"know" + 0.009*"anyone" + 0.008*"system" + 0.008*"memory"'), (19, '0.011*"jesus" + 0.005*"shall" + 0.005*"christ" + 0.005*"spirit" + 0.005*"koresh" + 0.005*"david" + 0.004*"father" + 0.004*"john" + 0.004*"matthew" + 0.004*"tobacco"')]

In [28]:
pip install pyLDAvis

In [30]:
pip install --upgrade gensim

     |████████████████████████████████| 23.9MB 1.5MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [10]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)